# <center>Statoil Kaggle competition. Part 3.</center>
## <center>Comparison of different Deep Learning models</center>

Models' weights have several hundred of MB size sometimes. Since I trained Deep Learning models on Google Colab website, this comparison was also created and executed there.

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import *
from fastai.metrics import error_rate

bs = 64  # batch size

In [3]:
from google.colab import drive
drive.mount('/content/drive')

### Copy train data and models from Google Drive

In [ ]:
# competition folder
!mkdir sample_data/statoil

!cp ../content/drive/'My Drive'/train.zip sample_data/statoil
!cp ../content/drive/'My Drive'/train_pseudo.zip sample_data/statoil

!unzip sample_data/statoil/train.zip -d sample_data/statoil/
!unzip sample_data/statoil/train_pseudo.zip -d sample_data/statoil/

# copy pseudo labeled images from the test dataset to the train folder
!cp sample_data/statoil/train_pseudo/band_1/*.jpg sample_data/statoil/train/band_1
!cp sample_data/statoil/train_pseudo/band_2/*.jpg sample_data/statoil/train/band_2

In [13]:
!mkdir sample_data/statoil/train/band_1/models
!mkdir sample_data/statoil/train/band_2/models

# copy models from Google Drive
!cp ../content/drive/'My Drive'/*-band-1.pth sample_data/statoil/train/band_1/models
!cp ../content/drive/'My Drive'/*-band-2.pth sample_data/statoil/train/band_2/models

In [7]:
def get_path(is_train, band_name):
    return 'sample_data/statoil/{}/{}/'.format('train' if is_train else 'test', band_name)

fnames = get_image_files(get_path(True, 'band_1'))
print(len(fnames))

3305


### Check models trained on band-1 data

In [14]:
# Models available
!ls -a sample_data/statoil/train/band_1/models

.   resnet101-band-1.pth  resnet50-band-1.pth
..  resnet34-band-1.pth   vgg19-band-1.pth


In [8]:
np.random.seed(2)
pat = r'/(\d)_[^/]+.jpg$'  # pattern for is_iceberg labels (0 or 1 at the beginning of a filename)

data = ImageDataBunch.from_name_re(get_path(True, 'band_1'),
                                   fnames,
                                   pat,
                                   ds_tfms=get_transforms(flip_vert=True, max_rotate=30., max_zoom=1.1),
                                   size=74,
                                   bs=bs).normalize()
# check number of classes
data.c

2

In [31]:
# download a model structure
vgg19 = cnn_learner(data, models.vgg19_bn, metrics=error_rate)

# load pretrained model weights
vgg19.load('vgg19-band-1')

# calculate confusion matrix
vgg19 = ClassificationInterpretation.from_learner(vgg19).confusion_matrix()

# the same for other models
resnet34 = cnn_learner(data, models.resnet34, metrics=error_rate)
resnet34.load('resnet34-band-1')
resnet34 = ClassificationInterpretation.from_learner(resnet34).confusion_matrix()

resnet50 = cnn_learner(data, models.resnet50, metrics=error_rate)
resnet50.load('resnet50-band-1')
resnet50 = ClassificationInterpretation.from_learner(resnet50).confusion_matrix()

resnet101 = cnn_learner(data, models.resnet101, metrics=error_rate)
resnet101.load('resnet101-band-1')
resnet101 = ClassificationInterpretation.from_learner(resnet101).confusion_matrix()

In [33]:
# produce a dataframe with results

models = ['vgg19', 'resnet34', 'resnet50', 'resnet101']
accuracy = []
precision = []
recall = []

for cm in [vgg19, resnet34, resnet50, resnet101]:
    accuracy.append(cm.trace() / cm.sum())
    precision.append(cm[1][1] / (cm[1][1] + cm[0][1]))
    recall.append(cm[1][1] / (cm[1][1] + cm[1][0]))

In [34]:
# show results

(pd.DataFrame()
 .assign(model = models,
         accuracy = accuracy,
         precision = precision,
         recall = recall)
)

,model,accuracy,precision,recall
0,vgg19,0.922844,0.936909,0.905488
1,resnet34,0.927383,0.932099,0.920732
2,resnet50,0.922844,0.910979,0.935976
3,resnet101,0.901664,0.887906,0.917683


### Check models trained on band-2 data

In [35]:
# Models available
!ls -a sample_data/statoil/train/band_2/models

.   resnet34-band-2.pth  vgg16-band-2.pth
..  resnet50-band-2.pth  vgg19-band-2.pth


In [36]:
np.random.seed(2)
pat = r'/(\d)_[^/]+.jpg$'  # pattern for is_iceberg labels (0 or 1 at the beginning of a filename)

data = ImageDataBunch.from_name_re(get_path(True, 'band_2'),
                                   fnames,
                                   pat,
                                   ds_tfms=get_transforms(flip_vert=True, max_rotate=30., max_zoom=1.1),
                                   size=74,
                                   bs=bs).normalize()
# check number of classes
data.c

2

In [39]:
# same as for band-1, but a bit different set of models

vgg16 = cnn_learner(data, models.vgg16_bn, metrics=error_rate)
vgg16.load('vgg16-band-2')
vgg16 = ClassificationInterpretation.from_learner(vgg16).confusion_matrix()

vgg19 = cnn_learner(data, models.vgg19_bn, metrics=error_rate)
vgg19.load('vgg19-band-2')
vgg19 = ClassificationInterpretation.from_learner(vgg19).confusion_matrix()

resnet34 = cnn_learner(data, models.resnet34, metrics=error_rate)
resnet34.load('resnet34-band-2')
resnet34 = ClassificationInterpretation.from_learner(resnet34).confusion_matrix()

resnet50 = cnn_learner(data, models.resnet50, metrics=error_rate)
resnet50.load('resnet50-band-2')
resnet50 = ClassificationInterpretation.from_learner(resnet50).confusion_matrix()

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/checkpoints/vgg16_bn-6c64b313.pth


In [40]:
# produce a dataframe with results

models = ['vgg_16', 'vgg19', 'resnet34', 'resnet50']
accuracy = []
precision = []
recall = []

for cm in [vgg16, vgg19, resnet34, resnet50]:
  accuracy.append(cm.trace() / cm.sum())
  precision.append(cm[1][1] / (cm[1][1] + cm[0][1]))
  recall.append(cm[1][1] / (cm[1][1] + cm[1][0]))

In [41]:
# show results

(pd.DataFrame()
 .assign(model = models,
         accuracy = accuracy,
         precision = precision,
         recall = recall)
)

,model,accuracy,precision,recall
0,vgg_16,0.698941,0.951049,0.414634
1,vgg19,0.720121,0.967320,0.451220
2,resnet34,0.767020,0.953125,0.557927
3,resnet50,0.724660,0.939759,0.475610


## Conclusion

All metrics for Band-1 and Band-2 images were calculated on the training dataset including pseudo labelled images from the test dataset.

On Band-1 images all four models showed similar results in terms of Precision and Recall. Best model on accuracy score - Resnet34, on Precision - VGG19 and on Recall - ResNet50.

On Band-2 images models behave differently. While Precision showed by models is abound 0.95, Recall is only about 0.45. Final accuracy therefore is only about 0.72. Best model on accuracy and recall scores - ResNet34, on precision - VGG19.

### <center>Band 1 results</center>

 | Model | Accuracy | Precision | Recall |
 |-------|----------|-----------|--------|
 | VGG19 | 0.923 | 0.937 | 0.905 |
 | ResNet34 | 0.927 | 0.932 | 0.921 |
 | ResNet50 | 0.923 | 0.911 | 0.936 |
 | ResNet101 | 0.902 | 0.888 | 0.918 |
 
### <center>Band 2 results</center>

 | Model | Accuracy | Precision | Recall |
 |-------|----------|-----------|--------|
 | VGG16 | 0.699 | 0.951 | 0.415 |
 | VGG19 | 0.720 | 0.967 | 0.451 |
 | ResNet34 | 0.767 | 0.953 | 0.558 |
 | ResNet50 | 0.725 | 0.940 | 0.476 |